# From Shouting Across a Room to the Repetition Code

Imagine you want to communicate with someone across a crowded, noisy room.  
The message you want to send is the word **“YES.”**

You shout **“YES!”** across the room.

But the room is loud and your friend hears something like **“M…ESS?”**  
Parts of the message are distorted or lost. This room is like a **noisy communication channel**.

To improve your chances of transmitting the correct message, you repeat the message many times:

> **“YES! YES! YES!”**

Now your friend hears:
- “YES, YES, M…ESS”

Even if one instance is unclear, the **majority** of what they hear is still “YES,” so they can confidently infer your message.

In digital communication, messages are sent as bits:
- `0` or `1`

Suppose you want to send the bit:
Instead of sending the bit once, you send it multiple times.

For example, a **3-bit repetition code**:
- `1` becomes `111`
- `0` becomes `000`

### Transmission with Noise
You send: 111
Noise corrupts one bit: 101

The receiver applies a simple rule:
- If most bits are `1`, decode as `1`
- If most bits are `0`, decode as `0`

For `101`:
- Two `1`s vs one `0`
- Decoded as `1`


Assume that at most one error occurs in each triplet of transmitted bits. In that case, the original message can be recovered using a **majority vote**: for example, the string `101` is decoded as `1`, while `100` is decoded as `0`.

The message $\bm{y} = \bm{x}\bm{x}\bm{x}$ sent through the channel is called an **encoding** of the original message $\bm{x}$. The process of reconstructing $\bm{x}$ from the noisy received message $\bm{\tilde{y}}$ is known as **decoding**. A specific encoding scheme is commonly referred to as a **code**, and the example described here is called the **3-repetition code**.

More generally, an error-correction procedure can be summarized by the following pipeline:

$$
\bm{x}
\;\xrightarrow{\;\mathcal{E}\;}\;
\bm{y}
\;\xrightarrow{\text{noise}}\;
\bm{\tilde{y}}
\;\xrightarrow{\;\mathcal{D}\;}\;
\hat{\bm{x}}
$$



## Encoder and Codewords

In general, an encoder $\mathcal{E}$ takes a message divided into blocks of $k$ bits and maps each block to a string of $n$ bits. We write

$$
\mathcal{E}(\bm{x}) = \bm{y}
$$

where $\bm{x}$ is a $k$-bit message and $\bm{y}$ is its $n$-bit encoding.

For the 3-repetition code, each single bit is encoded as three identical bits, so $k = 1$, $n = 3$, and

$$
\mathcal{E}(x) = xxx
$$

A **codeword** is any element in the image of $\mathcal{E}$. The set of all codewords is denoted by

$$
\mathcal{C} = \mathrm{Im}(\mathcal{E})
$$

In the 3-repetition code, the only codewords are `000` and `111`, corresponding to $\mathcal{E}(0)$ and $\mathcal{E}(1)$.

If one or two bits are flipped during transmission, the resulting string is no longer a valid codeword, and the error is therefore **detectable**. For instance, receiving `110` immediately signals that an error has occurred, since $110 \notin \mathcal{C}$. However, the error is only **correctable** by majority voting when at most one bit is flipped. If all three bits flip—transforming `000` into `111` or vice versa—the error becomes **undetectable**, leading to what is called a **logical error**.


## Distance 

This reasoning can be generalized using the notion of **distance**. The distance $d$ of a code is defined as the minimum number of bit flips required to transform one codeword into another. Equivalently, it is the smallest number of errors that can occur without being detected.

To formalize this idea, we introduce two related concepts:

- The **Hamming weight** $|\bm{x}|$ of a binary vector $\bm{x}$ is the number of entries equal to 1.
- The **Hamming distance** between two binary vectors $\bm{x}$ and $\bm{y}$, denoted $D_H(\bm{x}, \bm{y})$, is the number of positions at which they differ:

$$
D_H(\bm{x}, \bm{y}) = |\bm{x} - \bm{y}|
$$

Using this, the distance of a code is defined as

$$
d = \min_{\bm{x}, \bm{y} \in \mathcal{C}} D_H(\bm{x}, \bm{y})
$$

For the 3-repetition code, the distance is $d = 3$, since all three bits must be flipped to move from one codeword to the other.
A distance-d code can correct up to $\frac{d-1}{2}$ errors and detect up to $d-1$ errors. Why?

## Rate

A code that encodes $k$ bits into $n$ bits and has distance $d$ is called an $[n,k,d]$-code*. The 3-repetition code is therefore a $[3,1,3]$-code.

The **rate** of an $[n,k,d]$-code is defined as

$$
R = \frac{k}{n}
$$

In the case of the 3-repetition code, the rate is $$R = 1/3$$



## Trade-off

Designing effective error-correcting codes involves balancing **rate** and **distance**. A high rate implies low redundancy and minimal overhead, while a large distance allows the correction of many errors. Ideally, one would like both quantities to be large. Unfortunately, these goals are fundamentally in tension: increasing one typically requires sacrificing the other.


The basic idea behind **error correction** is that, within the vast space of all possible bit strings, only a restricted subset is considered **valid messages**. When a message is altered by noise during transmission, it is the receiver’s task to identify the most likely valid message that was originally sent.



## A 16-bit Message in a 4×4 Grid

Consider a 16-bit message arranged in a $4 \times 4$ grid:

$$
\begin{pmatrix}
0_0 & 1_1 & 0_2 & 1_3 \\
0_4 & 1_5 & 0_6 & 1_7 \\
1_8 & 1_9 & 0_{10} & 0_{11} \\
0_{12} & 0_{13} & 1_{14} & 1_{15}
\end{pmatrix}
$$

The subscripts indicate the **bit positions** in the flattened, row-wise representation of the message.

Rather than using all 16 bits to store information, we will be clever:  
some bits will store **data**, while others will store **redundancy** that allows us to detect (and later correct) errors.


## Parity as Redundancy

The redundancy we use is based on **parity**.

- The **parity** of a bit string is the number of `1`s it contains.
- If this number is even, the string has **even parity**.
- If it is odd, the string has **odd parity**.

We will store parity information in specific bits of the grid.

We choose the following parity checks:

- Bit at position **1** stores the parity of the **2nd and 4th columns**
- Bit at position **2** stores the parity of the **3rd and 4th columns**
- Bit at position **4** stores the parity of the **2nd and 4th rows**
- Bit at position **8** stores the parity of the **3rd and 4th rows**

For now, we ignore the bit at position 0.  
Although the placement of these parity bits may seem arbitrary, it is in fact highly structured, and this choice will become clear shortly.



## Example: A Column Parity Check

Let us explicitly compute the parity stored in **bit 1**, which checks the **2nd and 4th columns**.

The bits in the 2nd and 4th columns are:

- Column 2: $1_1, 1_5, 1_9, 0_{13}$
- Column 4: $1_3, 1_7, 0_{11}, 1_{15}$

Excluding the parity bit itself at position 1, we sum the remaining bits modulo 2:

$$
1_3 + 1_5 + 1_7 + 1_9 + 0_{11} + 0_{13} + 1_{15}
= 5
$$

Since the parity is **odd**, the parity bit must be set to `1`!


Each parity bit enforces a **constraint** on the allowed messages.  
A valid message must satisfy *all* parity checks simultaneously.

If a single bit flips during transmission, some parity checks will fail and the specific pattern of failed checks reveals where the error occurred.

## Why Is This Useful?

Assume now that the sender transmits a block that may contain **at most one error**. For example, suppose the receiver gets the following 4×4 block:

$$
\begin{pmatrix}
0_0 & 1_1 & 0_2 & 1_3 \\
0_4 & 1_5 & 1_6 & 1_7 \\
1_8 & 1_9 & 0_{10} & 0_{11} \\
0_{12} & 0_{13} & 1_{14} & 1_{15}
\end{pmatrix}
$$

Compared to the original message, the bit at position $6$ has flipped from `0` to `1`.



## Pinpointing the Error with Parity Checks

The receiver now recomputes the four parity checks.

In this example:
- The **first** and **fourth** parity checks are satisfied.
- The **second** and **third** parity checks are violated.

Each bit in the grid participates in a unique subset of parity checks.  
The pattern of failed checks therefore acts like an **address**:
- No error on columns 2 and 4
- Error on columns 3 and 4
- Error on rows 2 and 4
- No error on rows 3 and 4
Which combined singles out row 2 and column 3, that is **bit 6**.

The receiver can now **flip that bit back**, thereby correcting the error.

## What If the Error Is on a Parity Bit?

*Question*:
What happens if the error occurs on one of the parity check bits themselves?

*Solution*:
The same strategy still applies:
- A flipped parity bit causes **exactly one parity check** to fail.
- That failure pattern uniquely identifies the parity bit itself.
- The receiver flips it back, restoring consistency.

Thus, parity bits are protected by the same mechanism as data bits.



## The Special Role of Bit 0

At this point, there is a subtle issue with the bit at position \(0\).

By construction, if **no error has occurred**, the parity-check pattern points to position \(0\). This makes bit 0 ambiguous: is it actually in error, or is the block error-free?

There are two standard ways to deal with this.



## Option 1: Ignore Bit 0

We may simply discard bit 0 altogether.  
In this case:
- We have **15 physical bits**
- Of which **11 carry logical information**

This construction is known as the **\((15,11)\)-Hamming code**.



## Option 2: Use Bit 0 as a Global Parity Bit

Alternatively, we can use bit 0 to store the **parity of the entire block**, including all other bits.
This extended construction is called the **extended \((15,11)\)-Hamming code**.

*Question*: Which additional piece of information does this bit carries?

*Solution*: The global parity bit allows the receiver to **detect the presence of at least two errors**.  
While the code can still only *correct* one error, it can now reliably *detect* when two or more errors have occurred.


Notice that this scheme is remarkably efficient:
- Only **4 parity bits** are required to protect an **11-bit message**.
- In general, the number of parity bits needed scales as



*Question*: Can you provide the scaling of this scheme?
$$
\log_2(\text{number of bits})
$$



## General Form of a Hamming Code

A Hamming code with \(r\) parity bits has parameters

$$
(2^r - 1,\; 2^r - 1 - r),
$$

where:
- \(2^r - 1\) is the total number of physical bits
- \(2^r - 1 - r\) is the number of logical  bits

Hamming codes are the canonical example of **single-error-correcting, double-error-detecting** codes, and they illustrate how carefully placed redundancy enables reliable communication over noisy channels.


## Binary Indexing and Error Localization

One of the most elegant features of **Hamming codes** is that they are designed so that the outcomes of the parity checks, when read in **binary**, directly identify the location of a single error.

Let us label each position in our $4 \times 4$ block by its **binary index**, and assume again that a single error has occurred in our running example:

$$
\begin{pmatrix}
0_{0000} & 1_{0001} & 1_{0010} & 1_{0011} \\
0_{0100} & 1_{0101} & 0_{0110} & 1_{0111} \\
1_{1000} & 0_{1001} & 0_{1010} & 0_{1011} \\
0_{1100} & 0_{1101} & 1_{1110} & 1_{1111}
\end{pmatrix}
$$

Suppose that the parity checks corresponding to **bit positions 1 and 8** fail. Writing these indices in binary,

$$
1 = 0001, \qquad 8 = 1000
$$

and combining them gives

$$
0001 \oplus 1000 = 1001.
$$

This binary string points directly to the location of the erroneous bit: **position $1001$**.

In other words, the parity-check outcomes *spell out the address of the error in binary*.  
Isn’t that remarkably elegant?



## The XOR Function

Let us now turn this insight into a simple and powerful implementation strategy.

The key tool is the **exclusive OR (XOR)** operation, denoted $\oplus$.

For bits $a,b \in \{0,1\}$,
- $a \oplus b = 1$ if $a \neq b$,
- $a \oplus b = 0$ if $a = b$.

Crucially, XOR corresponds to **addition modulo 2**, which makes it perfectly suited for parity checks.



## Using XOR to Perform All Parity Checks at Once

Instead of computing many parity checks separately, we can proceed as follows:

- Take all bit positions whose value is `1`
- XOR their **binary indices** together

The responsibility of the sender is to ensure that, for a valid codeword,

$$
\bigoplus_{\text{positions } i \text{ with } x_i = 1} i = 0000.
$$

This single XOR condition encodes *all parity checks simultaneously*.



## Question  
**What happens if a single error occurs during transmission?**

### Solution  
If a bit flips (either \(0 \to 1\) or \(1 \to 0\)), the set of positions containing a `1` changes by exactly one index. As a result, the overall XOR becomes

$$
\bigoplus i = j,
$$

where \(j\) is precisely the binary index of the erroneous bit.

Thus, the XOR output directly identifies the error location, regardless of whether the bit flipped from `0` to `1` or from `1` to `0`.



## Question  
**Why do we choose the parity-bit positions the way we did?**

### Solution  
Parity bits are placed at positions that are **powers of two** (binary strings with a single `1`). This ensures that:

- Each parity check corresponds to exactly one binary digit
- Every data bit participates in a unique combination of parity checks
- The parity groups are **disjoint in a controlled way**

This disjunction guarantees that every single-bit error produces a **unique syndrome**, making the code both simple and optimal for single-error correction.




In [2]:
[1, 2] != [1, 3]

True